In [1]:
# !pip install spacy-transformers

In [2]:
# !pip install scispacy

In [3]:
# ! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_scibert-0.4.0.tar.gz

In [4]:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_lg-0.3.0.tar.gz

In [1]:
import scispacy
import spacy

In [2]:
from scispacy.linking import EntityLinker

In [3]:
# nlp = spacy.load("en_core_sci_scibert")
nlp = spacy.load("en_core_sci_lg")

/home/ubuntu/anaconda3/lib/python3.8/site-packages/spacy/util.py:715: UserWarning: [W094] Model 'en_core_sci_lg' (0.3.0) specifies an under-constrained spaCy version requirement: >=2.3.1. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.0.3,<3.1.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config.cfg from /home/ubuntu/anaconda3/lib/python3.8/site-packages/en_core_sci_lg/en_core_sci_lg-0.3.0/config.cfg

In [ ]:
linker = EntityLinker(name='umls', resolve_abbreviations=True)

In [ ]:
nlp.add_pipe(linker)

In [ ]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

In [ ]:
entity = doc.ents[1]

In [11]:
print("Name: ", entity)

Name:  bulbar muscular atrophy


In [13]:
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
    print(linker.kb.cui_to_entity[umls_ent[0]])

KeyError: "[E001] No component 'scispacy_linker' found in pipeline. Available names: ['transformer', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']"